In [1]:
from htm.bindings.sdr import SDR, Metrics
from htm.encoders.date import DateEncoder
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
import numpy as np
import pandas as pd
import pathlib
import datetime
import csv
from datetime import datetime
import os
from htm.encoders.rdse import RDSE, RDSE_Parameters
import time
import traceback
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn_extra.cluster import KMedoids
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

In [2]:
class ReflexiveMemory:
  def __init__(self, reflexSize):
    self.acKey0 = None
    self.pairs = {}
    self.anomalyRM = []
    self.anomalyTM = []
    self.enableLearn = False
    self.tableSize  = reflexSize
    self.historyRM = []
    self.historyTM = []
    self.historyGT = []

  def add(self, denseColumns):
    acKey1 = '-'.join(map(str, denseColumns.sparse))
    if(self.acKey0 != None):

      sequence = self.pairs.get(self.acKey0, {})
      sequence_data = sequence.get(acKey1, {
         "count": 0,
         "time": datetime.now()
      })
      sequence_data["count"] = sequence_data["count"] + 1
      sequence_data["time"] = datetime.now()

      if self.pairs.get(self.acKey0, None) is None:
        self.pairs[self.acKey0] = { acKey1: sequence_data }
      else:
        self.pairs[self.acKey0][acKey1] = sequence_data

    self.acKey0 = acKey1

  def predict(self, denseColumns):
    return_count = 0
    return_sdr = None

    acKey = '-'.join(map(str, denseColumns.sparse))
    sequences = self.pairs.get(acKey, {})
    for sequence_key, sequence_data in sequences.items():
      if sequence_data["count"] > return_count:
        return_count = sequence_data["count"]
        return_sdr = sequence_key

    if return_sdr is not None:
      tmp_sdr = SDR( denseColumns.dimensions )
      tmp_sdr.sparse = list(map(int, return_sdr.split('-')))
      return_sdr = tmp_sdr
    else:
      return_count = None

    return return_count, return_sdr

  def learn(self, denseColumns1, sp, tm):

    tm.activateDendrites(True)
    predictiveColumns = SDR( tm.getPredictiveCells().dimensions[0] )
    predictiveColumns.sparse = list(set(sorted(list(np.where(tm.getPredictiveCells().dense == 1)[0]))))

    activeColumns1 = SDR( predictiveColumns.dimensions )
    sp.compute(denseColumns1, False, activeColumns1)

    denseColumns0 = SDR( denseColumns1.dimensions )
    denseColumns0.sparse = list(map(int, self.acKey0.split('-')))
    activeColumns0 = SDR( predictiveColumns.dimensions )
    sp.compute(denseColumns0, False, activeColumns0)
 
    reflexiveCount, denseReflexiveColumns = self.predict(denseColumns0)
    reflexiveColumns = SDR( predictiveColumns.dimensions )
    if denseReflexiveColumns is not None:
      sp.compute(denseReflexiveColumns, False, reflexiveColumns)    

    overlapRM = 1 - np.count_nonzero((reflexiveColumns.dense & activeColumns1.dense)) / np.count_nonzero(activeColumns1.dense)
    self.anomalyRM.append( overlapRM )

    overlapTM = 1 - np.count_nonzero((predictiveColumns.dense & activeColumns1.dense)) / np.count_nonzero(activeColumns1.dense)
    self.anomalyTM.append( overlapTM )

    self.historyRM.append( reflexiveColumns.dense )
    self.historyTM.append( predictiveColumns.dense )
    self.historyGT.append( activeColumns1.dense )

  def compute(self, denseColumns, sp, tm):
    if self.acKey0 is not None:
      self.learn(denseColumns, sp, tm)  
    self.add(denseColumns)
  

In [3]:
inputSources = [
   "monthly_sp500_pca.csv",
   "weekly_dow_jones.csv",
   "weekly_nasdaq.csv",
   "weekly_sp500.csv",
   "monthly_vix_close.csv",
   "monthly_vix_high.csv",
   "monthly_vix_low.csv",
   "monthly_vix_open.csv",
   "daily_natural_gas.csv",
   "daily_oil_prices.csv",
   "value1_vix_close.csv",
   "value1_vix_high.csv",
   "value1_vix_low.csv",
   "value1_vix_open.csv"
]

In [4]:

config = {
    'enc': {
        "value" :
            {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
        "time": 
            {'timeOfDay': (30, 1), 'weekend': 21}
    },
    'sp': {
        'inputDimensions': None,
        'columnDimensions': (1638,),
        'potentialPct': 0.85,
        'potentialRadius': None,
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False,
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    },
    'anomaly': {'period': 1000},
    'learnRows': 100,
    'reflexSize': 128
}


In [5]:
def anomalyScore(y, x):
    if np.count_nonzero(y) != 0:
        return 1 - np.count_nonzero((x & y)) / np.count_nonzero(y)
    return 1

def match(y, x, idx1):
  overlap_min = 0.02
  n_samples = len(y)
  score1 = anomalyScore(y[idx1], x[idx1])
  if score1 > (1 - overlap_min):
     idx_closest = None
     score_closest = None
     for idx2 in range(n_samples):
        score2 = anomalyScore(y[idx2], x[idx1])
        if score_closest is None or score_closest > score2:
            score_closest = score2
            idx_closest = idx2
     return idx_closest
  return idx1

def roc_auc_score_multiclass(y_true, y_pred):
    scores = []
    for y_class in set(y_true):
        y_true_class = [True if x == y_class else False for x in y_true]
        y_pred_class = [True if x == y_class else False for x in y_pred]
        scores.append(roc_auc_score(y_true_class, y_pred_class))
    return sum(scores) / len(scores)


In [6]:
input_path = pathlib.Path('../datasets/numenta')
dataset_metrics = []

pbar = tqdm(total=len(inputSources))
for dataset in inputSources:

    tm_infer_tm = 0
    tm_infer_rm = 0

    records = []
    with open(input_path.joinpath(dataset), "r") as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        next(reader)
        next(reader)
        for record in reader:
            records.append(record)
        
    scalarEncoderParams = RDSE_Parameters()
    scalarEncoderParams.size = config["enc"]["value"]["size"]
    scalarEncoderParams.sparsity = config["enc"]["value"]["sparsity"]
    scalarEncoderParams.resolution = config["enc"]["value"]["resolution"]
    scalarEncoder = RDSE( scalarEncoderParams )
    encodingWidth = (scalarEncoder.size)

    config['sp']['inputDimensions'] = (encodingWidth,)
    config['sp']['potentialRadius'] = encodingWidth

    sp = SpatialPooler(
        inputDimensions = config['sp']['inputDimensions'],
        columnDimensions = config['sp']['columnDimensions'],
        potentialPct = config['sp']['potentialPct'],
        potentialRadius = config['sp']['potentialRadius'],
        globalInhibition = config['sp']['globalInhibition'],
        localAreaDensity = config['sp']['localAreaDensity'],
        synPermInactiveDec = config['sp']['synPermInactiveDec'],
        synPermActiveInc = config['sp']['synPermActiveInc'],
        synPermConnected = config['sp']['synPermConnected'],
        boostStrength = config['sp']['boostStrength'],
        wrapAround = config['sp']['wrapAround'],
        seed = config['sp']['seed']
    )

    tm = TemporalMemory(
        columnDimensions = config['sp']['columnDimensions'],
        cellsPerColumn = config['tm']['cellsPerColumn'],
        activationThreshold = config['tm']['activationThreshold'],
        initialPermanence = config['tm']['initialPermanence'],
        connectedPermanence = config['sp']['synPermConnected'],
        minThreshold = config['tm']['minThreshold'],
        maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
        permanenceIncrement = config['tm']['permanenceIncrement'],
        permanenceDecrement = config['tm']['permanenceDecrement'],
        predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
        maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
        maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
    )

    rm = ReflexiveMemory( config['reflexSize'] )

    enc_info = Metrics( [encodingWidth], 999999999)
    sp_info = Metrics( sp.getColumnDimensions(), 999999999 )
    tm_info = Metrics( [tm.numberOfCells()], 999999999 )
    anomaly_history = AnomalyLikelihood(config["anomaly"]["period"])

    inputs = []
    anomaly = []
    anomalyProb = []
   
    try:
        
        for count, record in enumerate(records):

            consumption = float(record[1])

            inputs.append( consumption )
            consumptionBits = scalarEncoder.encode(consumption)

            encoding = SDR( consumptionBits )
            enc_info.addData( encoding )
            
            activeColumns = SDR( sp.getColumnDimensions() )

            learn_sp = config['sp']['learn']
            learn_tm = config['tm']['learn']
            if count < config['learnRows']:
                learn_sp = True
                learn_tm = True

            rm.compute(encoding, sp, tm)

            sp.compute(encoding, learn_sp, activeColumns)
            sp_info.addData( activeColumns )

            tmp_tm = time.time()
            tm.compute(activeColumns, learn=learn_tm)
            tm_info.addData( tm.getActiveCells().flatten() )
            tm_infer_tm = tm_infer_tm + (time.time() - tmp_tm)

            tmp_tm = time.time()
            rm.predict(encoding)
            tm_infer_rm = tm_infer_rm + (time.time() - tmp_tm)

            anomaly.append( tm.anomaly )
            anomalyProb.append( anomaly_history.compute(tm.anomaly) )

    except Exception as e:
        print(traceback.format_exc())
        print(e)

    metric = {}
    metric['dataset'] = dataset

    n_samples = len(rm.historyGT)

    Y_labels = list(range(n_samples))
    X_labels = [ match(rm.historyGT, rm.historyRM, idx) for idx in range(n_samples)]
    precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
    metric['total-infe-time-rm'] = tm_infer_rm
    metric['infer-time-rm'] = tm_infer_rm / len(rm.anomalyRM)
    metric['anomaly-avg-rm'] = sum(rm.anomalyRM) / len(rm.anomalyRM)
    metric['anomaly-samples-rm'] = len(rm.anomalyRM)
    metric['accuracy0-rm'] = accuracy_score(Y_labels, X_labels)
    metric['precision0-rm'] = precision
    metric['recall0-rm'] = recall
    metric['fscore0-rm'] = fscore
    metric['support0-rm'] = support
    metric['auc0-rm'] = roc_auc_score_multiclass(Y_labels, X_labels)
    
    X = pairwise_distances(rm.historyGT, metric=anomalyScore)
    kmedoids = KMedoids(n_clusters=12, random_state=0).fit(X)
    X_labels = kmedoids.labels_
    Y = pairwise_distances(rm.historyRM, rm.historyGT, metric=anomalyScore)
    Y_labels = kmedoids.predict(Y)
    precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
    metric['accuracy1-rm'] = accuracy_score(Y_labels, X_labels)
    metric['precision1-rm'] = precision
    metric['recall1-rm'] = recall
    metric['fscore1-rm'] = fscore
    metric['support1-rm'] = support
    metric['auc1-rm'] = roc_auc_score_multiclass(Y_labels, X_labels)


    Y_labels = list(range(n_samples))
    X_labels = [ match(rm.historyGT, rm.historyTM, idx) for idx in range(n_samples)]
    precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
    metric['total-infer-time-tm'] = tm_infer_tm
    metric['infer-time-tm'] = tm_infer_tm / len(rm.anomalyTM)
    metric['anomaly-avg-tm'] = sum(rm.anomalyTM) / len(rm.anomalyTM)
    metric['anomaly-samples-tm'] = len(rm.anomalyTM)
    metric['accuracy0-tm'] = accuracy_score(Y_labels, X_labels)
    metric['precision0-tm'] = precision
    metric['recall0-tm'] = recall
    metric['fscore0-tm'] = fscore
    metric['support0-tm'] = support
    metric['auc0-tm'] = roc_auc_score_multiclass(Y_labels, X_labels)


    X = pairwise_distances(rm.historyGT, metric=anomalyScore)
    kmedoids = KMedoids(n_clusters=12, random_state=0).fit(X)
    X_labels = kmedoids.labels_
    Y = pairwise_distances(rm.historyTM, rm.historyGT, metric=anomalyScore)
    Y_labels = kmedoids.predict(Y)
    precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
    metric['accuracy1-tm'] = accuracy_score(Y_labels, X_labels)
    metric['precision1-tm'] = precision
    metric['recall1-tm'] = recall
    metric['fscore1-tm'] = fscore
    metric['support1-tm'] = support
    metric['auc1-tm'] = roc_auc_score_multiclass(Y_labels, X_labels)


    metric['nupic-anomaly-avg'] = sum(anomaly) / len(anomaly)
    metric['nupic-anomaly-prob-avg'] = np.count_nonzero(anomalyProb) / len(anomalyProb)

    dataset_metrics.append(metric)
    pbar.update(1)
    # break

pbar.close()

 57%|█████▋    | 8/14 [15:18<16:04, 160.80s/it]/usr/local/lib/python3.11/site-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 2 is empty! self.labels_[self.medoid_indices_[2]] may not be labeled with its corresponding cluster (2).
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 3 is empty! self.labels_[self.medoid_indices_[3]] may not be labeled with its corresponding cluster (3).
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 4 is empty! self.labels_[self.medoid_indices_[4]] may not be labeled with its corresponding cluster (4).
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn_extra/clu

In [7]:
df = pd.DataFrame(dataset_metrics)
df.to_csv('metrics.csv', index=False)

| Variable | Source | Description | 
| :---------------- | :------ | :------ | 
| dataset | | Dataset Name | 
| precision0-rm | RM | Precision (1-to-1 Matching) | 
| recall0-rm | RM | Recall (1-to-1 Matching) | 
| fscore0-rm | RM | F-Score (1-to-1 Matching) | 
| support0-rm | RM | Support (1-to-1 Matching) | 
| accuracy0-rm | RM | Accuracy (1-to-1 Matching) | 
| precision1-rm | RM | Precision (Clustering) | 
| recall1-rm | RM | Recall (Clustering) | 
| fscore1-rm | RM | F-Score (Clustering) | 
| support1-rm | RM | Support (Clustering) | 
| accuracy1-rm | RM | Accuracy (Clustering) | 
| anomaly-avg-rm | RM | Average Anomaly (Ilia Method) | 
| anomaly-samples-rm | RM | Samples | 
| total-infe-time-rm | RM | Inference Time per input | 
| infer-time-rm | RM | Total Inference Time (all samples) | 
| precision0-tm | TM | Precision (1-to-1 Matching) |
| recall0-tm | TM | Recall (1-to-1 Matching) |
| fscore0-tm | TM | F-Score (1-to-1 Matching) |
| support0-tm | TM | Support (1-to-1 Matching) |
| accuracy0-tm | TM | Accuracy (1-to-1 Matching) |
| precision1-tm | TM | Precision (Clustering) |
| recall1-tm | TM | Recall (Clustering) |
| fscore1-tm | TM | F-Score (Clustering) |
| support1-tm | TM | Support (Clustering) |
| accuracy1-tm | TM | Accuracy (Clustering) |
| anomaly-avg-tm | TM | Average Anomaly (Ilia Method) | 
| anomaly-samples-tm | TM | Samples | 
| total-infer-time-tm | TM | Inference Time per input | 
| infer-time-tm | TM | Total Inference Time (all samples) | 
| nupic-anomaly-avg | TM | Average Anomaly (Numenta Method) |
| nupic-anomaly-prob-avg | TM | Number of Anomalous Instances (Percentage) |


In [8]:
features = [
    'dataset',
    'total-infe-time-rm',
    'infer-time-rm',
    'anomaly-avg-rm',
    'anomaly-samples-rm',
    'accuracy0-rm',
    'precision0-rm',
    'recall0-rm',
    'fscore0-rm',
    'support0-rm',
    'auc0-rm',
    'accuracy1-rm',
    'precision1-rm',
    'recall1-rm',
    'fscore1-rm',
    'support1-rm',
    'auc1-rm'
]
df[features]

,dataset,total-infe-time-rm,infer-time-rm,anomaly-avg-rm,anomaly-samples-rm,accuracy0-rm,precision0-rm,recall0-rm,fscore0-rm,support0-rm,auc0-rm,accuracy1-rm,precision1-rm,recall1-rm,fscore1-rm,support1-rm,auc1-rm
0,monthly_sp500_pca.csv,0.015807,0.000010,0.804091,1641,0.513711,0.510970,0.513711,0.511580,None,0.756707,0.313224,0.344700,0.443914,0.347420,None,0.691294
1,weekly_dow_jones.csv,0.016704,0.000008,0.984501,2079,0.092833,0.091430,0.092833,0.091663,None,0.546198,0.096681,0.085831,0.151651,0.042081,None,0.531686
2,weekly_nasdaq.csv,0.018365,0.000009,0.907412,2080,0.290385,0.289664,0.290385,0.289744,None,0.645022,0.177404,0.139633,0.248097,0.126143,None,0.586397
3,weekly_sp500.csv,0.018515,0.000009,0.840728,2081,0.438251,0.437771,0.438251,0.437771,None,0.718990,0.262854,0.201477,0.316980,0.208375,None,0.624978
4,monthly_vix_close.csv,0.043652,0.000011,0.351807,4050,0.981975,0.981608,0.981975,0.981653,None,0.990985,0.571605,0.556711,0.563505,0.558398,None,0.762134
5,monthly_vix_high.csv,0.041862,0.000010,0.351115,4050,0.979012,0.978522,0.979012,0.978607,None,0.989504,0.628148,0.544298,0.515980,0.522746,None,0.740684
6,monthly_vix_low.csv,0.038902,0.000010,0.293378,4050,0.982469,0.982102,0.982469,0.982147,None,0.991232,0.668395,0.659318,0.677203,0.664682,None,0.822762
7,monthly_vix_open.csv,0.043370,0.000011,0.353669,4050,0.980247,0.979448,0.980247,0.979611,None,0.990121,0.592099,0.568046,0.564093,0.559264,None,0.763271
8,daily_natural_gas.csv,0.042409,0.000007,0.065013,5798,0.997068,0.996905,0.997068,0.996914,None,0.998534,1.000000,1.000000,1.000000,1.000000,None,NaN
9,daily_oil_prices.csv,0.070360,0.000008,0.214869,8300,0.981807,0.981688,0.981807,0.981688,None,0.990903,0.897349,0.872456,0.875193,0.873223,None,0.932886


In [9]:
features = [
    'dataset',
    'total-infer-time-tm',
    'infer-time-tm',
    'nupic-anomaly-avg',
    'anomaly-avg-tm',
    'nupic-anomaly-prob-avg',
    'anomaly-samples-tm',
    'accuracy0-tm',
    'precision0-tm',
    'recall0-tm',
    'fscore0-tm',
    'support0-tm',
    'auc0-tm',
    'accuracy1-tm',
    'precision1-tm',
    'recall1-tm',
    'fscore1-tm',
    'support1-tm',
    'auc1-tm'
]
df[features]

,dataset,total-infer-time-tm,infer-time-tm,nupic-anomaly-avg,anomaly-avg-tm,nupic-anomaly-prob-avg,anomaly-samples-tm,accuracy0-tm,precision0-tm,recall0-tm,fscore0-tm,support0-tm,auc0-tm,accuracy1-tm,precision1-tm,recall1-tm,fscore1-tm,support1-tm,auc1-tm
0,monthly_sp500_pca.csv,0.532506,0.000325,0.574570,0.574311,0.391596,1641,0.783668,0.768792,0.783668,0.773253,None,0.891768,0.349177,0.364157,0.427314,0.371598,None,0.683886
1,weekly_dow_jones.csv,0.630624,0.000303,0.935203,0.935172,0.519712,2079,0.371332,0.355981,0.371332,0.360577,None,0.685515,0.108225,0.099247,0.106035,0.077312,None,0.512348
2,weekly_nasdaq.csv,0.717278,0.000345,0.760198,0.760083,0.519942,2080,0.664904,0.646876,0.664904,0.652367,None,0.832371,0.221635,0.194494,0.209110,0.188438,None,0.569096
3,weekly_sp500.csv,0.665210,0.000320,0.617675,0.617492,0.520173,2081,0.816434,0.798937,0.816434,0.804457,None,0.908173,0.330130,0.281614,0.296844,0.281748,None,0.617803
4,monthly_vix_close.csv,1.249162,0.000308,0.070799,0.070569,0.753394,4050,0.993333,0.991867,0.993333,0.992292,None,0.996666,0.199506,0.232449,0.424747,0.184349,None,0.678122
5,monthly_vix_high.csv,1.065892,0.000263,0.072026,0.071797,0.753394,4050,0.992593,0.991372,0.992593,0.991713,None,0.996295,0.205432,0.241820,0.387154,0.151651,None,0.660008
6,monthly_vix_low.csv,0.760827,0.000188,0.034734,0.034496,0.753394,4050,0.996049,0.995086,0.996049,0.995354,None,0.998024,0.168642,0.319435,0.417395,0.207352,None,0.673662
7,monthly_vix_open.csv,1.145843,0.000283,0.070507,0.070278,0.753394,4050,0.992840,0.990842,0.992840,0.991434,None,0.996419,0.174321,0.222586,0.352166,0.160760,None,0.640885
8,daily_natural_gas.csv,0.315315,0.000054,0.012799,0.012629,0.827729,5798,0.998275,0.998122,0.998275,0.998137,None,0.999137,1.000000,1.000000,1.000000,1.000000,None,NaN
9,daily_oil_prices.csv,0.905460,0.000109,0.018910,0.018792,0.879653,8300,0.999036,0.998755,0.999036,0.998830,None,0.999518,0.541807,0.633425,0.707181,0.538869,None,0.833860
